In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Wang/3.Genome_wide_prediction/SCENT/SCENT_500.240604/pairs.E2G.res.rds"
path.matrix.atac_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Wang/K562/Kendall/atac_matrix.csv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Wang/1.prepare_data/1.seurat_pipeline.240507/rna_count_matrix.csv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Wang/3.Genome_wide_prediction/SCENT/SCENT_2500.240606/"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readRDS(path.pairs.E2G)

Import ATAC matrix

In [5]:
matrix.atac = read.csv(path.matrix.atac_count,
                       row.names = 1,
                       check.names = F)
matrix.atac = Matrix(as.matrix(matrix.atac), sparse = TRUE)
matrix.atac = BinarizeCounts(matrix.atac)

Import RNA matrix

In [6]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Prepare SCENT meta data

In [7]:
meta.data = data.frame(cell = colnames(matrix.rna),
                       nUMI = colSums(matrix.rna),
                       nMito = colSums(matrix.rna[grep("MT-",rownames(matrix.rna)),]),
                       celltype = "K562")
meta.data[,"log.nUMI"] = log(meta.data[,"nUMI"])
meta.data[,"percent.mito"] = meta.data[,"nMito"] / meta.data[,"nUMI"]
meta.data

,cell,nUMI,nMito,celltype,log.nUMI,percent.mito
,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
K562_Wang_AAACAGCCAATGCCCG-1,K562_Wang_AAACAGCCAATGCCCG-1,12561,794,K562,9.438352,0.063211528
K562_Wang_AAACAGCCACCCTCAC-1,K562_Wang_AAACAGCCACCCTCAC-1,25127,3064,K562,10.131698,0.121940542
K562_Wang_AAACAGCCACCTACTT-1,K562_Wang_AAACAGCCACCTACTT-1,18172,2314,K562,9.807637,0.127338763
K562_Wang_AAACATGCAACTAACT-1,K562_Wang_AAACATGCAACTAACT-1,14939,982,K562,9.611731,0.065733985
K562_Wang_AAACATGCAAGGTGCA-1,K562_Wang_AAACATGCAAGGTGCA-1,13074,1207,K562,9.478381,0.092320636
K562_Wang_AAACATGCACTAAATC-1,K562_Wang_AAACATGCACTAAATC-1,19262,1239,K562,9.865890,0.064323539
K562_Wang_AAACATGCAGCATGTC-1,K562_Wang_AAACATGCAGCATGTC-1,13940,2080,K562,9.542518,0.149210904
K562_Wang_AAACATGCATTAAGCT-1,K562_Wang_AAACATGCATTAAGCT-1,25225,3039,K562,10.135591,0.120475719
K562_Wang_AAACCAACAAATTGCT-1,K562_Wang_AAACCAACAAATTGCT-1,18599,2216,K562,9.830863,0.119146191


RUN SCENT prediction

In [8]:
SCENT_algorithm.modified = function (object, 
                                     # celltype, 
                                     # ncores, 
                                     # regr = "poisson", 
                                     # bin = TRUE) 
                                     regr = "poisson") {
    res <- data.frame()
    for (n in 1:nrow(object@peak.info)) {        
        gene <- object@peak.info[n, 1]
        this_peak <- object@peak.info[n, 2]
        atac_target <- data.frame(cell = colnames(object@atac), 
            atac = object@atac[this_peak, ])
        # if (bin) {
        #     atac_target[atac_target$atac > 0, ]$atac <- 1
        # }
        mrna_target <- object@rna[gene, ]
        df <- data.frame(cell = names(mrna_target), exprs = as.numeric(mrna_target))
        df <- merge(df, atac_target, by = "cell")
        df <- merge(df, object@meta.data, by = "cell")
        # df2 <- df[df[[object@celltypes]] == celltype, ]
        df2 <- df
        # nonzero_m <- length(df2$exprs[df2$exprs > 0])/length(df2$exprs)
        # nonzero_a <- length(df2$atac[df2$atac > 0])/length(df2$atac)
        # if (nonzero_m > 0.05 & nonzero_a > 0.05) {
            res_var <- "exprs"
            pred_var <- c("atac", object@covariates)
            formula <- as.formula(paste(res_var, paste(pred_var, 
                collapse = "+"), sep = "~"))
            if (regr == "poisson") {
                base = glm(formula, family = "poisson", data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_poisson
                assoc <- SCENT::assoc_poisson
            }
            else if (regr == "negbin") {
                base = glm.nb(formula, data = df2)
                coefs <- summary(base)$coefficients["atac", ]
                # assoc <- assoc_negbin
                assoc <- SCENT::assoc_negbin
            }
            # bs = boot::boot(df2, assoc, R = 100, formula = formula, 
            #     stype = "i", parallel = "no", ncpus = 1)
            # p0 = basic_p(bs$t0[1], bs$t[, 1])
            # p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # if (p0 < 0.1) {
            #     bs = boot::boot(df2, assoc, R = 500, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.05) {
                bs = boot::boot(df2, assoc, R = 2500, formula = formula, 
                  stype = "i", 
                  # parallel = "multicore", ncpus = ncores)
                  parallel = "no", ncpus = 1)
                # p0 = basic_p(bs$t0[1], bs$t[, 1])
                p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.01) {
            #     bs = boot::boot(df2, assoc, R = 25000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            # if (p0 < 0.001) {
            #     bs = boot::boot(df2, assoc, R = 50000, formula = formula, 
            #       stype = "i", 
            #       # parallel = "multicore", ncpus = ncores)
            #       parallel = "no", ncpus = 1)
            #     # p0 = basic_p(bs$t0[1], bs$t[, 1])
            #     p0 = SCENT::basic_p(bs$t0[1], bs$t[, 1])
            # }
            out <- data.frame(gene = gene, peak = this_peak, 
                beta = coefs[1], se = coefs[2], z = coefs[3], 
                p = coefs[4], boot_basic_p = p0)           
            res <- rbind(res, out)
        # }
    }
    object@SCENT.result <- res
    return(object)
}

In [9]:
dir.create(paste(dir.output,"chr",sep = "/"),recursive = T)

In [10]:
chr.done = dir(paste(dir.output,"chr",sep = "/"))
chr.run = as.character(unique(seqnames(pairs.E2G)))
chr.run = chr.run[!chr.run %in% chr.done]

In [11]:
for(chr.tmp in chr.run){
  print(chr.tmp)
  pairs.E2G.chr = pairs.E2G[seqnames(pairs.E2G) == chr.tmp]
  pairs.E2G.chr.res = pairs.E2G.chr[!is.na(pairs.E2G.chr$boot_basic_p) & pairs.E2G.chr$boot_basic_p < 0.05]
  pairs.E2G.chr.keep = pairs.E2G.chr[!pairs.E2G.chr %in% pairs.E2G.chr.res]
  gene_peak.chr = as.data.frame(mcols(pairs.E2G.chr.res)[,c("TargetGene","PeakName")])
  list.gene_peak.chr <- split(gene_peak.chr, seq_len(n.cores))
  
  list.obj.SCENT.chr <- lapply(list.gene_peak.chr, function(gene_peak.tmp) {
    obj.SCENT.tmp <- CreateSCENTObj(
      rna = matrix.rna[rownames(matrix.rna) %in% gene_peak.tmp$TargetGene,],
      atac = matrix.atac[rownames(matrix.atac) %in% gene_peak.tmp$PeakName,], 
      meta.data = meta.data,
      peak.info = gene_peak.tmp,
      covariates = c("log.nUMI","percent.mito"), 
      celltypes = "celltype"
    )
  })
  
  cl <- makeCluster(n.cores)
  clusterExport(cl, varlist=c("SCENT_algorithm.modified"))
  list.SCENT.result.chr <- parLapply(cl, list.obj.SCENT.chr, function(obj.SCENT.tmp) {
    obj.SCENT.tmp <- SCENT_algorithm.modified(object = obj.SCENT.tmp)
    return(obj.SCENT.tmp@SCENT.result)
  })
  stopCluster(cl)
  
  
  df.SCENT.result.chr = do.call(rbind,list.SCENT.result.chr)
  rownames(df.SCENT.result.chr) = paste(df.SCENT.result.chr$peak,
                                        df.SCENT.result.chr$gene,
                                        sep = "_")

  pairs.E2G.chr.res = pairs.E2G.chr.res[pairs.E2G.chr.res$PairName %in% rownames(df.SCENT.result.chr)]
  mcols(pairs.E2G.chr.res)[,c("beta","se","z","p","boot_basic_p")] = 
    df.SCENT.result.chr[pairs.E2G.chr.res$PairName,c("beta","se","z","p","boot_basic_p")]
  pairs.E2G.chr.res$boot_basic_p.log10 = -log10(pairs.E2G.chr.res$boot_basic_p)
  pairs.E2G.chr.res$sign = 1
  pairs.E2G.chr.res$sign[pairs.E2G.chr.res$beta < 0] = -1
  pairs.E2G.chr.res$boot_basic_p.log10.signed = pairs.E2G.chr.res$boot_basic_p.log10 * pairs.E2G.chr.res$sign
  pairs.E2G.chr.merge = c(pairs.E2G.chr.res,
                          pairs.E2G.chr.keep)
  saveRDS(pairs.E2G.chr.merge,
          paste(dir.output,"chr",chr.tmp,sep = "/"))
  rm (pairs.E2G.chr.merge)
}

[1] "chr1"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr10"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr11"


...

Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chr9"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


[1] "chrX"


Warning message in split.default(x = seq_len(nrow(x)), f = f, drop = drop, ...):
“data length is not a multiple of split variable”


Merge results

In [12]:
chr.merge = as.character(unique(seqnames(pairs.E2G)))
list.res = list()
for(chr.tmp in chr.merge){
    list.res[[chr.tmp]] = readRDS(paste(dir.output,"chr",chr.tmp,sep = "/"))
}
pairs.E2G.res = unlist(as(list.res, "GRangesList"))

Save results

In [13]:
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
pairs.E2G.res

GRanges object with 122134 ranges and 11 metadata columns:
                 seqnames              ranges strand |  TargetGene
                    <Rle>           <IRanges>  <Rle> | <character>
  chr1.chr1.chr1     chr1 100037588-100039066      * |         DBT
  chr1.chr1.chr1     chr1 100037588-100039066      * |       PALMD
  chr1.chr1.chr1     chr1 100037588-100039066      * |       SASS6
  chr1.chr1.chr1     chr1 100132326-100133471      * |      CDC14A
  chr1.chr1.chr1     chr1 100132326-100133471      * |     SLC35A3
             ...      ...                 ...    ... .         ...
  chrX.chrX.chrX     chrX     9419399-9421119      * |       TBL1X
  chrX.chrX.chrX     chrX     9462929-9463416      * |       TBL1X
  chrX.chrX.chrX     chrX     9464643-9465695      * |       TBL1X
  chrX.chrX.chrX     chrX     9515625-9516599      * |       TBL1X
  chrX.chrX.chrX     chrX     9530800-9531331      * |       TBL1X
                               PeakName               PairName       b

In [14]:
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,100037588,100039066,DBT,K562,-0.08068284,0.03401389,-2.372055,0.0176894367,0.0320,1.494850,-1,-1.494850
chr1,100037588,100039066,PALMD,K562,-0.15710744,0.05895333,-2.664946,0.0077000708,0.0352,1.453457,-1,-1.453457
chr1,100037588,100039066,SASS6,K562,0.06276029,0.02513262,2.497165,0.0125190746,0.0424,1.372634,1,1.372634
chr1,100132326,100133471,CDC14A,K562,-0.05172105,0.01534895,-3.369681,0.0007525524,0.0128,1.892790,-1,-1.892790
chr1,100132326,100133471,SLC35A3,K562,0.10239522,0.03834137,2.670620,0.0075711340,0.0064,2.193820,1,2.193820
chr1,1001784,1002126,DVL1,K562,0.13429439,0.06493325,2.068191,0.0386220222,0.0336,1.473661,1,1.473661
chr1,100249481,100250305,FRRS1,K562,0.06048995,0.02346044,2.578381,0.0099264556,0.0424,1.372634,1,1.372634
chr1,100265575,100267022,RTCA-AS1,K562,0.14751292,0.06718326,2.195680,0.0281148669,0.0432,1.364516,1,1.364516
chr1,100265575,100267022,SASS6,K562,0.06808660,0.02791037,2.439474,0.0147086781,0.0648,1.188425,1,1.188425


In [15]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
[1] Matrix_1.6-5         GenomicRanges_1.54.1 GenomeInfoDb_1.38.1 
[4] IRanges_2.36.0       S4Vectors_0.40.2     BiocGeneri